### thanks to this person for part of the code
https://www.kaggle.com/dschettler8845/captk-brats-preprocessing-cleaned-commented

In [ ]:
import os
import sys 
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import glob
from pathlib import Path
from skimage.transform import resize
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import SimpleITK as sitk
import re
import subprocess
import multiprocessing
from scipy import ndimage as ndi
import shutil
import tarfile
import skimage.transform as skTrans
train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'

In [ ]:
# installation captk 
# установка captk

In [ ]:
def install_captk(package_input_dir="/kaggle/input/captk-181-installerbin", working_dir="/kaggle/working"):
    print("\n\n... INSTALLING BC PACKAGE ...\n\n")
    os.system('apt install bc')
    
    print("\n\n... MOVING THE BIN PACKAGE TO WORKING DIRECTORY AND MODIFYING PERMISSIONS ...\n\n")
    !cp {os.path.join(package_input_dir, "CaPTk_1.8.1_Installer.bin")} {working_dir}
    !chmod +x {os.path.join(working_dir, "CaPTk_1.8.1_Installer.bin")}
    
    print("\n\n... INSTALLING CAPTK ...\n\n")
    !echo -e Y | {os.path.join(working_dir, "CaPTk_1.8.1_Installer.bin")}
    
    print("\n\n... EXTRACT CONTENTS OF APPIMAGE ONTO HARD DRIVE ...\n\n")
    subprocess.call([os.path.join(working_dir, "CaPTk/1.8.1/captk"), "--appimage-extract"])
    
    # Add relevant directories to the respective paths
    print("\n\n... ADD RELEVANT PATHS TO SYSTEM PATHS ...\n\n")
    os.environ['PATH'] = os.path.join(working_dir, "squashfs-root/usr/lib:") + os.environ['PATH'] 
    os.environ['LD_LIBRARY_PATH'] = os.path.join(working_dir, "squashfs-root/usr/lib:") + os.environ['LD_LIBRARY_PATH'] 
    
    print("\n\n... SEE CAPTK COMMAND HELP [-h] ...\n\n")
    !{os.path.join(working_dir, "squashfs-root/usr/bin/Preprocessing")} -h
    
install_captk()

In [ ]:
# dicom to nifti conversion
# конвертация dicom в nifti

In [ ]:
train_dirs = os.listdir(train_path)

In [ ]:
reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

In [ ]:
if not os.path.isdir('./flair'):
    os.mkdir('./flair')
for i, folder in enumerate(tqdm(train_dirs)): #train_dirs[:100]
    train_study = train_dirs[i]
    filenamesDICOM = sorted(reader.GetGDCMSeriesFileNames(f'{train_path}{train_study}/FLAIR'), key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    reader.SetFileNames(filenamesDICOM)
    flair = reader.Execute()
    sitk.WriteImage(flair, f'./flair/test_{train_study}_flair.nii')

In [ ]:
# if not os.path.isdir('./t1w'):
#     os.mkdir('./t1w')
# for i, folder in enumerate(tqdm(train_dirs)):
#     train_study = train_dirs[i]
#     filenamesDICOM = sorted(reader.GetGDCMSeriesFileNames(f'{train_path}{train_study}/T1w'), key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
#     reader.SetFileNames(filenamesDICOM)
#     flair = reader.Execute()
#     sitk.WriteImage(flair, f'./t1w/test_{train_study}_t1.nii.gz')

In [ ]:
# if not os.path.isdir('./t1ce'):
#     os.mkdir('./t1ce')
# for i, folder in enumerate(tqdm(train_dirs)):
#     train_study = train_dirs[i]
#     filenamesDICOM = sorted(reader.GetGDCMSeriesFileNames(f'{train_path}{train_study}/T1wCE'), key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
#     reader.SetFileNames(filenamesDICOM)
#     flair = reader.Execute()
#     sitk.WriteImage(flair, f'./t1ce/test_{train_study}_t1ce.nii.gz')

In [ ]:
# if not os.path.isdir('./t2w'):
#     os.mkdir('./t2w')
# for i, folder in enumerate(tqdm(train_dirs)):
#     train_study = train_dirs[i]
#     filenamesDICOM = sorted(reader.GetGDCMSeriesFileNames(f'{train_path}{train_study}/T2w'), key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
#     reader.SetFileNames(filenamesDICOM)
#     flair = reader.Execute()
#     sitk.WriteImage(flair, f'./t2w/test_{train_study}_t2.nii.gz')

In [ ]:
# регистрация nifti по образцу  brats 20
# registration of nifti on the model of brats 20

In [ ]:
flags={'-rME': 'NMI', '-rIS':'1', '-rNI':'100,50,5', '-reg': 'Affine'}
captk = './squashfs-root/usr/bin/Preprocessing'
fixed_file = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001/BraTS20_Training_001_flair.nii'

In [ ]:
# only flair
# только flair

In [ ]:
if not os.path.isdir('./flairreg'):
    os.mkdir('./flairreg')
for i, file in enumerate(tqdm(os.listdir('./flair'))):
    out = f'./flairreg/{file}'
    filepath = f'./flair/{file}'
    #command = f"{captk} -i {file} -rFI {fixed_file} -o ./flairreg/{file}.gz -reg Affine -rIS {flags['-rIS']}"
    res = subprocess.call([captk, '-i', file, '-rFI', fixed_file, '-o', out,'-rME', flags['-rME'], '-reg', flags['-reg'], '-rIS', flags['-rIS']])
    print("Returned Value: ", res)
    if res==0:
        os.remove(filepath)
    else:
        print('error')

In [ ]:
# only t1ce
# только t1ce

In [ ]:
if not os.path.isdir('./t1cereg'):
    os.mkdir('./t1cereg')
for i, file in enumerate(tqdm(os.listdir('./t1ce'))):
    out = f'./t1cereg/{file}'
    filepath = f'./t1ce/{file}'
    #command = f"{captk} -i {file} -rFI {fixed_file} -o ./flairreg/{file}.gz -reg Affine -rIS {flags['-rIS']}"
    res = subprocess.call([captk, '-i', file, '-rFI', fixed_file, '-o', out,'-rME', flags['-rME'], '-reg', flags['-reg'], '-rIS', flags['-rIS']])
    print("Returned Value: ", res)
    if res==0:
        os.remove(filepath)
    else:
        print('error')

In [ ]:
# convert and register flair at the same time
# и конвертация и регистрация flair одновременно

In [ ]:
if not os.path.isdir('./flair'):
    os.mkdir('./flair')
if not os.path.isdir('./flairreg'):
    os.mkdir('./flairreg')
#for i, folder in enumerate(tqdm(train_dirs)):
for i in range(len(sub_df)):
    train_study = sub_df.iloc[i]['BraTS21ID']
    filenamesDICOM = sorted(reader.GetGDCMSeriesFileNames(f'{train_path}{train_study}/FLAIR'), key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    reader.SetFileNames(filenamesDICOM)
    flair = reader.Execute()
    sitk.WriteImage(flair, f'./flair/test_{train_study}_flair.nii.gz')
    file = f'./flair/test_{train_study}_flair.nii.gz'
    oup = f'./flairreg/test_{train_study}_flair.nii.gz'
    res = subprocess.call([captk, '-i', file, '-rFI', fixed_file, '-o', oup,'-rME', flags['-rME'], '-reg', flags['-reg'], '-rIS', flags['-rIS']])
    print(f"{i} {file} Returned Value: ", res)
    if res==0:
        os.remove(file)
    else:
        print('error')

In [ ]:
# convert and register t1ce at the same time
# и конвертация и регистрация t1ce одновременно

In [ ]:
if not os.path.isdir('./t1ce'):
    os.mkdir('./t1ce')
if not os.path.isdir('./t1cereg'):
    os.mkdir('./t1cereg')
for i, folder in enumerate(tqdm(train_dirs)):
    train_study = train_dirs[i]
    filenamesDICOM = sorted(reader.GetGDCMSeriesFileNames(f'{train_path}{train_study}/T1wCE'), key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    reader.SetFileNames(filenamesDICOM)
    flair = reader.Execute()
    sitk.WriteImage(flair, f'./t1ce/test_{train_study}_t1ce.nii.gz')
    file = f'./t1ce/test_{train_study}_t1ce.nii.gz'
    oup = f'./t1cereg/test_{train_study}_t1ce.nii.gz'
    res = subprocess.call([captk, '-i', file, '-rFI', fixed_file, '-o', oup,'-rME', flags['-rME'], '-reg', flags['-reg'], '-rIS', flags['-rIS']])
    print(f"{file} Returned Value: ", res)
    if res==0:
        os.remove(file)
    else:
        print('error')

In [ ]:
!zip -r flairreg.zip ./flairreg

In [ ]:
from IPython.display import FileLink
FileLink('./flairreg.zip')